In [33]:
import pandas as pd

# Load the dataset into a pandas DataFrame
df = pd.read_csv('car data.csv')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('car data.csv')

# Check for missing values
print(df.isnull().sum())  # Check for any missing values

df = pd.get_dummies(df, drop_first=True)

scaler = StandardScaler()
df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven']] = scaler.fit_transform(df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven']])

# Display the cleaned data
df.head()


Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64


,Year,Selling_Price,Present_Price,Kms_Driven,Owner,Car_Name_Activa 3g,Car_Name_Activa 4g,Car_Name_Bajaj ct 100,Car_Name_Bajaj Avenger 150,Car_Name_Bajaj Avenger 150 street,...,Car_Name_swift,Car_Name_sx4,Car_Name_verna,Car_Name_vitara brezza,Car_Name_wagon r,Car_Name_xcent,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Individual,Transmission_Manual
0,0.123437,-0.259616,-0.237377,-0.256152,0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
1,-0.220587,0.016165,0.220357,0.155963,0,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,True
2,1.155509,0.508630,0.256280,-0.773871,0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
3,-0.908635,-0.358109,-0.404247,-0.817658,0,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,True
4,0.123437,-0.013383,-0.089048,0.141796,0,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True


In [35]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv('car data.csv')
df = df.drop('Car_Name', axis=1)

categorical_cols = ['Fuel_Type', 'Seller_Type', 'Transmission']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


numerical_cols = ['Year', 'Present_Price', 'Kms_Driven', 'Owner']
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Split data into features (X) and target (y)
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

# Check data types after preprocessing
print("Data types of X (features) after preprocessing:")
print(X.dtypes)

X = X.astype('float32')
y = y.astype('float32')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check data types after splitting
print("Data types of X_train after splitting:")
print(X_train.dtypes)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Check the data types of tensors to make sure they are correct
print("Data types of tensors:")
print(X_train_tensor.dtype, X_test_tensor.dtype, y_train_tensor.dtype, y_test_tensor.dtype)

# Define the neural network model
import torch.nn as nn
import torch.optim as optim

class CarPricePredictor(nn.Module):
    def __init__(self):
        super(CarPricePredictor, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
model = CarPricePredictor()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 1000
for epoch in range(epochs):
    model.train()


    y_pred = model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)

# Convert predictions to numpy arrays for comparison
y_pred = y_pred_tensor.numpy()
y_test = y_test_tensor.numpy()

# Calculate metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R²):", r2)


Data types of X (features) after preprocessing:
Year                      float64
Present_Price             float64
Kms_Driven                float64
Owner                     float64
Fuel_Type_Diesel             bool
Fuel_Type_Petrol             bool
Seller_Type_Individual       bool
Transmission_Manual          bool
dtype: object
Data types of X_train after splitting:
Year                      float32
Present_Price             float32
Kms_Driven                float32
Owner                     float32
Fuel_Type_Diesel          float32
Fuel_Type_Petrol          float32
Seller_Type_Individual    float32
Transmission_Manual       float32
dtype: object
Data types of tensors:
torch.float32 torch.float32 torch.float32 torch.float32
Epoch [100/1000], Loss: 4.134117126464844
Epoch [200/1000], Loss: 1.4341278076171875
Epoch [300/1000], Loss: 0.7944415807723999
Epoch [400/1000], Loss: 0.4980429708957672
Epoch [500/1000], Loss: 0.3964275121688843
Epoch [600/1000], Loss: 0.34748315811157227
Epoc